# 서울 지도 시각화
 - 서울 어린이 인구수
 - 서울 어린이 보호소
 - 서울 어린이 사고다발지역
 - 서울 스쿨존 내 어린이 사고다발지역

## 데이터 불러오기

In [ ]:
pip install geopandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from shapely.geometry import Polygon
import json
import requests
from folium import plugins
from folium.plugins import MarkerCluster

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 서울 어린이 보호 구역 CPZ (Children Protection Zone)
CPZ = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/서울시 어린이보호구역.csv', index_col = 0)

# 서울 어린이 인구수 NSC (Number Of Seoul Children)
NSC = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/서울시 어린이 인구 현황.csv', index_col = 0)

# 서울 어린이 사고다발지역 WCA (Walking Child Accident)
WCA = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/어린이 사고다발지역.csv', index_col = 0)

# 서울 스쿨존 내 어린이 사고다발지역 (SchoolZoon Child Accident)
SCA = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/스쿨존내어린이 사고다발지역정보.csv', index_col = 0)

# 서울 단속 카메라 위치 (Seoul Control Camera)
SCC = pd.read_csv('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/Data/서울시무인교통단속카메라(전체).csv', index_col = 0, encoding='cp949')

data = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
content = data.content
seoul_data = json.loads(content)

In [ ]:
NSC=NSC.set_index('행정구역')
NSC.replace(',', '', regex=True, inplace=True)
NSC = NSC.apply(pd.to_numeric)
NSC = NSC.drop('서울특별시')

In [ ]:
array = ['' for x in range(25)]
for i in range(25):
  array[i] = seoul_data['features'][i]['properties']['name']
NSC = NSC.reindex(index=array)

In [ ]:
# Json은 int를 이해하지 못해

# seoul_data['features']는 리스트
# seoul_data['features'][0] 는 딕
# seoul_data['features'][0]['properties'] 는 딕
for i in range(25):
  seoul_data['features'][i]['properties']['children_population'] = str(NSC['2021년 어린이 인구수'][i])[0:-3]+','+str(NSC['2021년 어린이 인구수'][i])[-3:]+'명'
  seoul_data['features'][i]['properties']['total_population'] = str(NSC['2021년 총 인구수'][i])[0:-3]+','+str(NSC['2021년 총 인구수'][i])[-3:]+'명'
  seoul_data['features'][i]['properties']['children_ratio'] = f"{NSC['2021년 어린이 인구수'][i] / NSC['2021년 총 인구수'][i] *100:.2f}%"

In [ ]:
WCA.reset_index(inplace=True)
WCA

,index,법정동코드,지점코드,연도,행정구역,지점명,발생건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,위도,경도,다발지역폴리곤
0,85,1117012900,11170001,2016,용산구,"서울특별시 용산구 이촌동(이촌1동 주민선터, 용산청소년수 부근)",3,4,0,0,2,2,37.521424,126.973010,"{""type"":""Polygon"",""coordinates"":[[[126.9748069..."
1,86,1123010600,11230001,2016,동대문구,서울특별시 동대문구 장안동(승용빌딩 부근),3,3,0,1,1,1,37.566494,127.068121,"{""type"":""Polygon"",""coordinates"":[[[127.0699180..."
2,87,1126010200,11260001,2016,중랑구,서울특별시 중랑구 상봉동(중화초교 부근),3,3,0,0,3,0,37.602417,127.094933,"{""type"":""Polygon"",""coordinates"":[[[127.0967298..."
3,88,1126010400,11260002,2016,중랑구,서울특별시 중랑구 묵동(묵2동사무소 부근),3,3,0,0,3,0,37.609448,127.074558,"{""type"":""Polygon"",""coordinates"":[[[127.0763544..."
4,89,1129013600,11290001,2016,성북구,서울특별시 성북구 하월곡동(하월곡생명중앙교회 부근),3,4,0,0,4,0,37.607851,127.036376,"{""type"":""Polygon"",""coordinates"":[[[127.0381721..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,147,1168010600,11680001,2020,강남구,서울특별시 강남구 대치동(대치사거리 부근),3,4,0,0,4,0,37.502885,127.058485,"{""type"":""Polygon"",""coordinates"":[[[127.0602818..."
63,148,1171011300,11710001,2020,송파구,서울특별시 송파구 거여동(거마로11 부근),3,5,0,1,2,2,37.494416,127.144762,"{""type"":""Polygon"",""coordinates"":[[[127.1465583..."
64,149,1171010100,11710002,2020,송파구,서울특별시 송파구 잠실동(신천역 부근),3,3,0,0,2,1,37.511498,127.085444,"{""type"":""Polygon"",""coordinates"":[[[127.0872404..."
65,150,1174010700,11740001,2020,강동구,서울특별시 강동구 암사동(암사2동주민센터 부근),4,5,0,0,5,0,37.551906,127.126900,"{""type"":""Polygon"",""coordinates"":[[[127.1286964..."


In [ ]:
# WCA 폴리곤 내 위도와 경도 순서변경
array_WCA = ['' for x in range(len(WCA['다발지역폴리곤']))]
for i in range(len(WCA['다발지역폴리곤'])):
  array_WCA[i] = json.loads(WCA['다발지역폴리곤'][i])
  for j in range(len(array_WCA[i]['coordinates'][0])):
    array_WCA[i]['coordinates'][0][j][0], array_WCA[i]['coordinates'][0][j][1] = array_WCA[i]['coordinates'][0][j][1], array_WCA[i]['coordinates'][0][j][0]

In [ ]:
# SCA 폴리곤 내 위도와 경도 순서변경
array_SCA = ['' for x in range(len(SCA['다발지역폴리곤']))]
for i in range(len(SCA['다발지역폴리곤'])):
  array_SCA[i] = json.loads(SCA['다발지역폴리곤'][i])
  for j in range(len(array_SCA[i]['coordinates'][0])):
    array_SCA[i]['coordinates'][0][j][0], array_SCA[i]['coordinates'][0][j][1] = array_SCA[i]['coordinates'][0][j][1], array_SCA[i]['coordinates'][0][j][0]

In [ ]:
center = [37.541, 126.986]  # 지도의 중심 좌표,여기 예는 서울시 좌표입니다. 
seoul_map = folium.Map(location=center,
                       tiles='cartodbpositron',
                       zoom_start=11, # seoul_map변수 지정, 서울시, 화면 해상도는 11
                       control_scale=True)

choropleth=folium.Choropleth(geo_data = seoul_data,
                  data=NSC['2021년 어린이 인구수'],
                  columns=['name', 'children_population'],
                  fill_color = 'YlOrBr',
                  fill_opacity = 0.5,
                  line_opacity = 1,
                  name = '2021년 어린이 인구수',
                  key_on='feature.properties.name',
                  legend_name='2021년 어린이 인구수(명)'
).add_to(seoul_map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['name', 'total_population', 'children_population', 'children_ratio'], aliases=['행정구역: ', '2021년 총 인구수: ', '2021년 어린이 인구수: ', '어린이 인구 비율'], labels=True)
)

fg = folium.FeatureGroup(name='어린이 보호구역')
seoul_map.add_child(fg)

fggg = folium.FeatureGroup(name='어린이 사고 다발지역')
seoul_map.add_child(fggg)

fgggg = folium.FeatureGroup(name='스쿨존 내 어린이 사고 다발지역')
seoul_map.add_child(fgggg)

fggggg = folium.FeatureGroup(name='단속카메라 설치 위치')
seoul_map.add_child(fggggg)

# SubGroup 생성시
g1 = plugins.FeatureGroupSubGroup(fg, '어린이 보호구역 서브그룹')
seoul_map.add_child(g1)

g2 = plugins.FeatureGroupSubGroup(fg, '어린이 보호구역 클러스터 서브그룹')
seoul_map.add_child(g2)
marker_cluster = MarkerCluster().add_to(g2)

for i in CPZ.index:
  folium.Circle(
      location =[CPZ['위도'][i], CPZ['경도'][i]],
      radius = 5,
      color = 'blue',
      fill_color = 'blue',
      fill_opacity = 100,
      tooltip = CPZ['대상시설명'][i]
).add_to(g1)

for i in CPZ.index:
  folium.Circle(
      location =[CPZ['위도'][i], CPZ['경도'][i]],
      radius = 5,
      color = 'blue',
      fill_color = 'blue',
      fill_opacity = 100,
      tooltip = CPZ['대상시설명'][i]
).add_to(marker_cluster)

for i in range(len(WCA['다발지역폴리곤'])):
  folium.Polygon(
      locations = array_WCA[i]['coordinates'][0],
      fill = True,
      weight = 3,
      opacity = 1,
      color = '#FF0000',
      fill_color = 'red',
      tooltip = WCA['지점명'][i]
  ).add_to(fggg)

for i in range(len(SCA['다발지역폴리곤'])):
  folium.Polygon(
      locations = array_SCA[i]['coordinates'][0],
      fill = True,
      weight = 3,
      opacity = 1,
      color = '#FF00FF',
      fill_color = 'magenta',
      tooltip = WCA['지점명'][i]
  ).add_to(fgggg)

for i in SCC.index:
  folium.Circle(
      location =[SCC['위도'][i], SCC['경도'][i]],
      radius = 5,
      color = 'green',
      fill_color = 'green',
      fill_opacity = 100,
      tooltip = SCC['설치장소'][i]
).add_to(fggggg) 

folium.LayerControl().add_to(seoul_map)

seoul_map

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
center = [37.541, 126.986]  # 지도의 중심 좌표,여기 예는 서울시 좌표입니다. 
seoul_map = folium.Map(location=center,
                       zoom_start=11, # seoul_map변수 지정, 서울시, 화면 해상도는 11
                       control_scale=True)

choropleth=folium.Choropleth(geo_data = seoul_data,
                  data=NSC['2021년 어린이 인구수'],
                  columns=['name', 'children_population'],
                  fill_color = 'YlOrBr',
                  fill_opacity = 0.5,
                  line_opacity = 1,
                  name = '2021년 어린이 인구수',
                  key_on='feature.properties.name',
                  legend_name='2021년 어린이 인구수(명)'
).add_to(seoul_map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['name', 'total_population', 'children_population', 'children_ratio'], aliases=['행정구역: ', '2021년 총 인구수: ', '2021년 어린이 인구수: ', '어린이 인구 비율'], labels=True)
)
fg = folium.FeatureGroup(name='어린이 보호구역')
seoul_map.add_child(fg)

fgg = folium.FeatureGroup(name='어린이 보호 구역 클러스터')
seoul_map.add_child(fgg)
marker_cluster = MarkerCluster().add_to(fgg)

fggg = folium.FeatureGroup(name='어린이 사고 다발지역')
seoul_map.add_child(fggg)

fgggg = folium.FeatureGroup(name='스쿨존 내 어린이 사고 다발지역')
seoul_map.add_child(fgggg)

fggggg = folium.FeatureGroup(name='단속카메라 설치 위치')
seoul_map.add_child(fggggg)

for i in CPZ.index:
  folium.Circle(
      location =[CPZ['위도'][i], CPZ['경도'][i]],
      radius = 5,
      color = 'blue',
      fill_color = 'blue',
      fill_opacity = 100,
      tooltip = CPZ['대상시설명'][i]
).add_to(fg)

for i in CPZ.index:
  folium.Circle(
      location =[CPZ['위도'][i], CPZ['경도'][i]],
      radius = 5,
      color = 'blue',
      fill_color = 'blue',
      fill_opacity = 100,
      tooltip = CPZ['대상시설명'][i]
).add_to(marker_cluster)

for i in range(len(WCA['다발지역폴리곤'])):
  folium.Polygon(
      locations = array_WCA[i]['coordinates'][0],
      fill = True,
      weight = 3,
      opacity = 1,
      color = '#FF0000',
      fill_color = 'red',
      tooltip = WCA['지점명'][i]
  ).add_to(fggg)

for i in range(len(SCA['다발지역폴리곤'])):
  folium.Polygon(
      locations = array_SCA[i]['coordinates'][0],
      fill = True,
      weight = 3,
      opacity = 1,
      color = '#FF00FF',
      fill_color = 'magenta',
      tooltip = SCA['지점명'][i]
  ).add_to(fgggg)

for i in SCC.index:
  folium.Circle(
      location =[SCC['위도'][i], SCC['경도'][i]],
      radius = 5,
      color = 'green',
      fill_color = 'green',
      fill_opacity = 100,
      tooltip = SCC['설치장소'][i]
).add_to(fggggg) 

folium.LayerControl().add_to(seoul_map)

seoul_map

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
seoul_map.save('/content/drive/MyDrive/[D9&10] 데이터시각화 프로젝트_2조/seoul_map.html')

In [ ]:
center = [37.541, 126.986]  # 지도의 중심 좌표,여기 예는 서울시 좌표입니다. 
seoul_map = folium.Map(location=center,
                       zoom_start=11, # seoul_map변수 지정, 서울시, 화면 해상도는 11
                       control_scale=True,
                       width='70%',
                       height='70%')

choropleth=folium.Choropleth(geo_data = seoul_data,
                  data=NSC['2021년 어린이 인구수'],
                  columns=['name', 'children_population'],
                  fill_color = 'YlOrBr',
                  fill_opacity = 0.5,
                  line_opacity = 1,
                  name = '2021년 어린이 인구수',
                  key_on='feature.properties.name',
                  legend_name='2021년 어린이 인구수(명)'
).add_to(seoul_map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['name', 'total_population', 'children_population', 'children_ratio'], aliases=['행정구역: ', '2021년 총 인구수: ', '2021년 어린이 인구수: ', '어린이 인구 비율'], labels=True)
)
fg = folium.FeatureGroup(name='어린이 보호구역')
seoul_map.add_child(fg)

fgg = folium.FeatureGroup(name='어린이 보호 구역 클러스터')
seoul_map.add_child(fgg)
marker_cluster = MarkerCluster().add_to(fgg)

fggg = folium.FeatureGroup(name='어린이 사고 다발지역')
seoul_map.add_child(fggg)

fgggg = folium.FeatureGroup(name='스쿨존 내 어린이 사고 다발지역')
seoul_map.add_child(fgggg)

fggggg = folium.FeatureGroup(name='단속카메라 설치 위치')
seoul_map.add_child(fggggg)

for i in CPZ.index:
  folium.Circle(
      location =[CPZ['위도'][i], CPZ['경도'][i]],
      radius = 300,
      color = 'blue',
      fill_color = 'blue',
      fill_opacity = 0,
      tooltip = CPZ['대상시설명'][i]
).add_to(fg)

for i in CPZ.index:
  folium.Circle(
      location =[CPZ['위도'][i], CPZ['경도'][i]],
      radius = 5,
      color = 'blue',
      fill_color = 'blue',
      fill_opacity = 100,
      tooltip = CPZ['대상시설명'][i]
).add_to(marker_cluster)

for i in range(len(WCA['다발지역폴리곤'])):
  folium.Polygon(
      locations = array_WCA[i]['coordinates'][0],
      fill = True,
      weight = 3,
      opacity = 1,
      color = '#FF0000',
      fill_color = 'red',
      tooltip = WCA['지점명'][i]
  ).add_to(fggg)

for i in range(len(SCA['다발지역폴리곤'])):
  folium.Polygon(
      locations = array_SCA[i]['coordinates'][0],
      fill = True,
      weight = 3,
      opacity = 1,
      color = '#FF00FF',
      fill_color = 'magenta',
      tooltip = SCA['지점명'][i]
  ).add_to(fgggg)

for i in SCC.index:
  folium.Circle(
      location =[SCC['위도'][i], SCC['경도'][i]],
      radius = 5,
      color = 'green',
      fill_color = 'green',
      fill_opacity = 100,
      tooltip = SCC['설치장소'][i]
).add_to(fggggg) 

folium.LayerControl().add_to(seoul_map)

seoul_map

Output hidden; open in https://colab.research.google.com to view.